In [1]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wntr
import scipy as sc
import networkx as nx
import networkx.drawing.nx_pylab as nxp
from pipedream_solver.hydraulics import SuperLink
from pipedream_solver.simulation import Simulation
from pipedream_solver.nutils import interpolate_sample
import random
import time
import pickle
import pipedream_utility as pdu
from pipedream_utility import *
import pipedream_simulation as pd_sim
from pipedream_simulation import *
import pipedream_simulation_sensor_results as pd_sim_sensor
from pipedream_simulation_sensor_results import *
from pipedream_kalman_filter import apply_EKF
import viswaternet as vis
import matplotlib as mpl

#Don't show future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
images_dir = 'Images/'

In [2]:
%matplotlib inline

In [3]:
#Parameter for how many hours to run the model
t_run=24

# Time step and number of iterations
dt_list = [60,300,600,900,1800]
iter_list = [1,5,10,20,40]

# Networks
net_list = ['Networks/Net2.inp', 'Networks/CA1.inp', 'Networks/PA1.inp'] 

# Initialize empty dictionary to store results
dict_results = {}

In [4]:
for inp in net_list:
    dict_results_inter = {}
    for num in range(10):    
        dt_list_time = []
        dt_list_error = []
        for dt in dt_list:
            it_list_time = []
            it_list_error = []
            for num_iter in iter_list:
                if inp == 'Networks/Net2.inp':
                    name = 'NET2'
                    H_net2_pd, Q_net2_pd, Q_pump, model, Q_in_all_df, pd_time = run_pipedream_simulation(inp, t_run = t_run, dt = dt, banded = False, num_iter = num_iter)
                    
                    wn = wntr.network.WaterNetworkModel(inp)
                    wn.options.time.report_timestep=dt    
                    wn.options.time.duration=t_run*3600
                    sim = wntr.sim.EpanetSimulator(wn)
                    
                    t1 = time.time()
                    results = sim.run_sim()
                    t2 = time.time()
                    wntr_time = t2-t1
                    
                    H_net2_wn = results.node['head']
                    Q_net2_wn= results.link['flowrate']
                    
                    abs_diff_node_net2 = (H_net2_pd[H_net2_wn.columns] - H_net2_wn[H_net2_wn.columns]).abs().mean(axis=0)
                    abs_diff_link_net2 = (Q_net2_pd[Q_net2_wn.columns] - Q_net2_wn[Q_net2_wn.columns]).abs().mean(axis=0)
                    
                    it_list_time.append(pd_time/wntr_time)
                    it_list_error.append(abs_diff_node_net2.mean())
                    
                if inp == 'Networks/CA1.inp':
                    axnum = 1
                    name = 'CA1'
                    H_ca1_pd, Q_ca1_pd, Q_pump, model, Q_in_all_df, pd_time = run_pipedream_simulation(inp, t_run = t_run, dt = dt, banded = False, num_iter = num_iter)
                    
                    wn = wntr.network.WaterNetworkModel(inp)
                    wn.options.time.report_timestep=dt    
                    wn.options.time.duration=t_run*3600
                    sim = wntr.sim.EpanetSimulator(wn)
                    
                    t1 = time.time()
                    results = sim.run_sim()
                    t2 = time.time()
                    wntr_time = t2-t1
                    
                    H_ca1_wn = results.node['head']
                    Q_ca1_wn= results.link['flowrate']
                    
                    abs_diff_node_ca1 = (H_ca1_pd[H_ca1_wn.columns] - H_ca1_wn[H_ca1_wn.columns]).abs().mean(axis=0)
                    abs_diff_link_ca1 = (Q_ca1_pd[Q_ca1_wn.columns] - Q_ca1_wn[Q_ca1_wn.columns]).abs().mean(axis=0)

                    it_list_time.append(pd_time/wntr_time)
                    it_list_error.append(abs_diff_node_ca1.mean())
                    
                if inp == 'Networks/PA1.inp':
                    axnum = 2
                    name = 'PA1'
                    H_pa1_pd, Q_pa1_pd, Q_pump, model, Q_in_all_df, pd_time = run_pipedream_simulation(inp, t_run = t_run, dt = dt, banded = False, num_iter = num_iter)
                    
                    wn = wntr.network.WaterNetworkModel(inp)
                    wn.options.time.report_timestep=dt    
                    wn.options.time.duration=t_run*3600
                    sim = wntr.sim.EpanetSimulator(wn)
                    
                    t1 = time.time()
                    results = sim.run_sim()
                    t2 = time.time()
                    wntr_time = t2-t1
                    
                    H_pa1_wn = results.node['head']
                    Q_pa1_wn= results.link['flowrate']
                    
                    abs_diff_node_pa1 = (H_pa1_pd[H_pa1_wn.columns] - H_pa1_wn[H_pa1_wn.columns]).abs().mean(axis=0)
                    abs_diff_link_pa1 = (Q_pa1_pd[Q_pa1_wn.columns] - Q_pa1_wn[Q_pa1_wn.columns]).abs().mean(axis=0)
                    
                    it_list_time.append(pd_time/wntr_time)
                    it_list_error.append(abs_diff_node_pa1.mean())
            
            dt_list_time.append(it_list_time)
            dt_list_error.append(it_list_error)
            
        net_times = pd.DataFrame(dt_list_time, index = dt_list, columns = iter_list)
        net_errors = pd.DataFrame(dt_list_error, index = dt_list, columns = iter_list)
        dict_results_inter[num] = {'times': net_times, 'errors': net_errors}
    dict_results[name] =  dict_results_inter
    

In [5]:
times_concat = pd.concat([dict_results[name][i]['times'] for i in range(10) for name in ['NET2', 'CA1', 'PA1']])
times_concat = times_concat.groupby(times_concat.index)
times_mean = times_concat.mean()
times_median = times_concat.median()
times_pa1 = pd.concat([dict_results['PA1'][i]['times'] for i in range(10)])
times_pa1_mean = times_pa1.groupby(times_pa1.index).mean()

error_concat = pd.concat([dict_results[name][i]['errors'] for i in range(10) for name in ['NET2', 'CA1', 'PA1']])
error_concat = error_concat.groupby(error_concat.index)
error_mean = error_concat.mean()
error_median = error_concat.median()
error_pa1 = pd.concat([dict_results['PA1'][i]['errors'] for i in range(10)])
error_pa1_mean = error_pa1.groupby(error_pa1.index).mean()

## Table 2. Comparison between PipeDream and EPANET run times.

In [6]:
times_mean

,1,5,10,20,40
60,13.198038,24.177393,23.876443,24.571350,23.954066
300,5.911988,14.794756,19.030993,17.318931,17.254199
600,3.601086,8.585185,10.847544,12.489999,12.169513
900,2.464454,6.030531,8.873588,9.799620,9.720222
1800,1.541632,3.804069,5.802206,7.039447,7.219720


In [13]:
error_mean

,1,5,10,20,40
60,0.021216,0.022604,0.022604,0.022604,0.022604
300,0.035069,0.029505,0.029524,0.029525,0.029525
600,0.058688,0.043617,0.043706,0.043708,0.043708
900,0.082802,0.059944,0.060103,0.060104,0.060106
1800,0.154324,0.114635,0.114927,0.114952,0.114953
